# Dysfunctional seed dispersal in the endemic flora of Madagascar

* Language: R
* Methods: linear regression, multivariate imputation by chained equations (MICE), spatial analyses
* Associated publication: (in prep)

Now, we want to estimate the number of available dispersers (lemurs only) by matching the __distribution of plant species__ to the __distribution of their potential dispersers__.

For this purpose, we are going to work with __rasters__. What's a raster? <br>
In its simplest form, a raster consists of a matrix of cells (or pixels) organized into rows and columns (or a grid) where each cell contains a value representing information, such as temperature. Rasters are digital aerial photographs, imagery from satellites, digital pictures, or even scanned maps. (definition from http://desktop.arcgis.com/en/arcmap/10.3/manage-data/raster-and-images/what-is-raster-data.htm)
<br>
<br>

In [3]:
library(raster);

<br>
<br>
## Create rasters for vegetation formations and lemurs

### Plant traits

In [4]:
# Load data
traitveg<-read.csv("PT-data-miceP.csv")

# Prepare a dataset with the variables needed: Seed width, the various vegetation formations (For, Bush, Grass, Anthr), and whether a plant is endozoochorous or not (SyndAB2)
traitveg$Veg_Bush2 <- ifelse(traitveg$Veg_Bush=="yes"|traitveg$Veg_Shrub=="yes","yes","no") #combine Shrubland with Bushland (to have a match with the vegetation formations in the vegetation map) 
traitveg <- traitveg[,c(".imp",".id", "SeWi", "Veg_Forest","Veg_Bush2","Veg_Grass","Veg_Anthr", "SyndAB2")]
colnames(traitveg) <- c(".imp", ".id", "SeWi", "For", "Bush", "Grass", "Anthr", "SyndAB2")
traitveg<-split(traitveg, list(traitveg$.imp))
traitveg<-traitveg[-1]
str(traitveg)

List of 20
 $ 1 :'data.frame':	8784 obs. of  8 variables:
  ..$ .imp   : int [1:8784] 1 1 1 1 1 1 1 1 1 1 ...
  ..$ .id    : Factor w/ 8784 levels "Abrahamia_buxifolia",..: 1 2 3 4 5 6 7 8 9 10 ...
  ..$ SeWi   : num [1:8784] 10.3 8 12.7 15.3 13.5 ...
  ..$ For    : Factor w/ 2 levels "no","yes": 2 2 2 1 2 2 2 2 1 2 ...
  ..$ Bush   : chr [1:8784] "no" "no" "no" "yes" ...
  ..$ Grass  : Factor w/ 2 levels "no","yes": 1 1 1 1 1 1 1 1 1 1 ...
  ..$ Anthr  : Factor w/ 2 levels "no","yes": 1 1 1 1 1 1 2 1 1 1 ...
  ..$ SyndAB2: Factor w/ 2 levels "Abiotic","Biotic": 2 2 2 2 2 2 2 2 2 2 ...
 $ 2 :'data.frame':	8784 obs. of  8 variables:
  ..$ .imp   : int [1:8784] 2 2 2 2 2 2 2 2 2 2 ...
  ..$ .id    : Factor w/ 8784 levels "Abrahamia_buxifolia",..: 1 2 3 4 5 6 7 8 9 10 ...
  ..$ SeWi   : num [1:8784] 10.3 8 12.7 15.3 6 ...
  ..$ For    : Factor w/ 2 levels "no","yes": 2 2 2 1 2 2 2 2 2 2 ...
  ..$ Bush   : chr [1:8784] "no" "no" "no" "yes" ...
  ..$ Grass  : Factor w/ 2 levels "no","yes": 

In [ ]:
# For each of the 20 imputed datasets, create a file with the data (this is to avoid subsequent memory issues)
for (d in 1:20) {
namefilev <- paste("traitveg", d, ".csv", sep="")
write.csv(traitveg[[d]], file=namefilev, row.names=TRUE)
}

We have to download points of occurrence of Madagascan plant species (i.e. locations where individuals of a given species have been observed) on GBIF http://www.gbif.org/occurrence/search?TAXON_KEY=6&COUNTRY=MG.

In [ ]:
# Load points of occurrence of Madagascan plant species
dat <- read.table("Plant_occurrences.txt", sep="\t", header=TRUE, fill=TRUE,quote = "")	
dat <- dat[,c("species", "decimalLatitude", "decimalLongitude")]

# replace " " by "_" in species name to match species names in our datasets
dat$species <- gsub(" ", "_", dat$species)            

# remove species with no distribution data
dat <- subset(dat, decimalLatitude != "NA" & decimalLongitude != "NA")      

# Save distribution data
write.table(dat, file="Plant_coord.txt", sep="\t")

<br>
### Vegetation formations

###### 1) Create rasters for each vegetation type

We first download the vegetation map on http://www.vegmad.org/datasets_gis.html.<br>

In [4]:
## Check the map information
cat0 <- read.csv("TIF-VegetationTypesValuesOriginal.csv", sep=";")
head(cat0)

VALUE,COUNT,RED,GREEN,BLUE,OPACITY,NAME12,CHAPTER12,WEBSITE_N1,ORDER2,RECLASS1
0,975559657,1.00,1.00,1.00,1,Background,<NA>,Unknown,20,0
1,4039415,0.25,0.88,0.82,1,Water,<NA>,12 Water,17,0
2,6774651,0.83,0.83,0.83,1,Bare Soil/Rock,<NA>,12 Bare Soil/Rock,16,0
3,2893166,0.93,0.51,0.93,1,Mangroves,5.14,7 Mangroves,11,3
4,27969115,0.63,0.13,0.94,1,Cultivation,5.15,11 Anthropic&Cultivated Areas,15,0
5,31235887,0.65,0.16,0.16,1,Western dry forest,5.8,1 Western Dry Forest,3,5


There are too many vegetation types (26). We don't need highly accurate vegetation types so we will combine some of them by adding new values to the file. For example, the vegetation types 7, 11, 12 and 19 are all different kinds of bushes so we assign an identical new value (4) to all of them.

In [14]:
cat <- read.csv("TIF-VegetationTypesValues.csv", sep=",")
cat

VALUE,COUNT,RED,GREEN,BLUE,OPACITY,NAME12,CHAPTER12,WEBSITE_N1,ORDER2,RECLASS1,Lemurs,PlantTrait_types,New.value,New.valueFor,New.valueBush,New.valueGrass,New.valueAnthr,New.valueBase
0,975559657,1.00,1.00,1.00,1,Background,<NA>,Unknown,20,0,0,NA,NA,0,0,0,0,0
1,4039415,0.25,0.88,0.82,1,Water,<NA>,12 Water,17,0,0,NA,NA,0,0,0,0,0
2,6774651,0.83,0.83,0.83,1,Bare Soil/Rock,<NA>,12 Bare Soil/Rock,16,0,0,NA,NA,0,0,0,0,0
3,2893166,0.93,0.51,0.93,1,Mangroves,5.14,7 Mangroves,11,3,0,Veg_Mang,NA,0,0,0,0,0
4,27969115,0.63,0.13,0.94,1,Cultivation,5.15,11 Anthropic&Cultivated Areas,15,0,0,Veg_Anthr,1,0,0,0,1,0
5,31235887,0.65,0.16,0.16,1,Western dry forest,5.8,1 Western Dry Forest,3,5,1,Veg_Forest,4,4,0,0,0,0
6,293326191,0.82,0.70,0.55,1,Plateau grassland-wooded grassland mosaic,5.5,6/3a Plateau grassland-wooded grassland mosaic,14,0,0,Veg_Grass,2,0,0,2,0,0
7,161401940,0.95,0.95,0.86,1,Wooded grassland-bushland,5.4,6/5 Wooded grassland-bushland,13,0,0,Veg_Bush2,3,0,3,0,0,0
9,85762,0.25,0.88,0.82,1,Western humid forest,5.7,1 Western humid forest,4,14,1,Veg_Forest,4,4,0,0,0,0
10,6777964,0.65,0.16,0.16,1,Western dry forest,5.8,1 Western Dry Forest,1,5,1,Veg_Forest,4,4,0,0,0,0


Now, we have a file giving the correspondence between old and new values for each vegetation type. <br>
The new values will be our raster's values.

The vegetation map combines all vegetation formations and it is huge (1.7 Go). So, to avoid subsequent memory issues, we want to create one raster per vegetation formation. Thus, when we need only one vegetation formation, we don't have to read the huge vegetation map.

In [ ]:
vegtype <- c("For","Grass","Anthr","Bush")

## Read raster from vegetation map (UTM zone 38 North, spheroid/datum WGS 84)
rasterveg <- raster("vegetation.tif")

## Create base map
mBase <- matrix(c(cat$VALUE, cat$New.valueBase),nrow=21, ncol=2)
rasterBase <- reclassify(rasterveg,mBase)       
writeRaster(rasterBase, filename="rasterBase.tif",format="GTiff", overwrite=TRUE) 

## Subset raster according to each vegetation type
mFor <- matrix(c(cat$VALUE, cat$New.valueFor),nrow=21, ncol=2)
rasterFor <- reclassify(rasterveg,mFor)        # Creates a new raster map whose category values are based upon a reclassification of the categories in an existing raster map (i.e. we use the new values instead of the old ones), here only taking into account Forests (other types=0)
writeRaster(rasterFor, filename="rasterFor.tif",format="GTiff", overwrite=TRUE) 

mBush <- matrix(c(cat$VALUE, cat$New.valueBush),nrow=21, ncol=2)
rasterBush <- reclassify(rasterveg,mBush)
writeRaster(rasterBush, filename="rasterBush.tif",format="GTiff", overwrite=TRUE) 

mGrass <- matrix(c(cat$VALUE, cat$New.valueGrass),nrow=21, ncol=2)
rasterGrass <- reclassify(rasterveg,mGrass)
writeRaster(rasterGrass, filename="rasterGrass.tif",format="GTiff", overwrite=TRUE) 

mAnthr <- matrix(c(cat$VALUE, cat$New.valueAnthr),nrow=21, ncol=2)
rasterAnthr <- reclassify(rasterveg,mAnthr)
writeRaster(rasterAnthr, filename="rasterAnthr.tif",format="GTiff", overwrite=TRUE) 

<br>
###### 2) Create rasters for each combination of vegetation types

In [ ]:
matveg <- expand.grid(For=c("no","yes"), Bush=c("no","yes"), Grass=c("no","yes"), Anthr=c("no","yes")) # combination matrix
matveg <- matveg[-1,]           # remove first line (with "no" only)

for (c in 1:nrow(matveg)) {
	combveg <- names(matveg[, matveg[c,]=="yes", drop=FALSE]) 	# vegetation types in combination c
	nameraster <- NULL
	listrast <- list()

	## Create a list with, as elements, the rasters of vegetation types (in the combination c)
	for (v in seq_along(combveg)) {
		nameraster[v] <- paste("raster",combveg[v], sep="")  # create name of the corresponding raster (vector of names)
		listrast[[v]] <- get(nameraster[v])                  # get the raster based on the character string of its name (list of rasters)
	}

	## When more than 1 raster in the list, stack them and write the corresponding combination raster
	if (length(listrast)>1) {
		stackrast <- stack (listrast, quick=TRUE)
		rasterTot <- overlay(stackrast, fun=sum,unstack=TRUE)    # raster combining all vegetation types included in 'combveg'
		combname <- paste(combveg, collapse="")
		namerastervegtif <- paste("raster",combname, ".tif", sep="") 
		writeRaster(rasterTot, filename=namerastervegtif,format="GTiff", overwrite=TRUE)
	}	
}

<br>
###### 3) Decrease resolution of each raster (from 29 m² to 1015 m² cells) (otherwise memory issues)

In [ ]:
for (c in 1:nrow(matveg)) {
	combveg <- names(matveg[, matveg[c,]=="yes", drop=FALSE]) 
	combname <- paste(combveg, collapse="")
	namerasterveg <- paste("raster", combname, sep="")
	namerastervegtif <- paste(namerasterveg, ".tif", sep="") 
	rasterveg <- raster(namerastervegtif)	
	rastervegagg <- aggregate(rasterveg, fact=35, fun=modal)             # to have 1015 m² cells
	namerasteraggvegtif <- paste("raster",combname, "Agg.tif", sep="") 
	writeRaster(rastervegagg, filename=namerasteraggvegtif,format="GTiff", overwrite=TRUE)
}


nameBase <- "rasterBase.tif"
rasterBase <- raster(nameBase)
rasterbasagg <- aggregate(rasterBase, fact=35, fun=modal)
writeRaster(rasterbasagg, filename="rasterBaseAgg.tif",format="GTiff", overwrite=TRUE)

###### 4) Reproject every vegetation raster to longlat

In [ ]:
rasterBase <- rasterbasagg

proj4string(rasterBase) <- CRS("+proj=utm +zone=38 +south +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0")   # original data in UTM

# reproject to longlat without keeping values (projectExtent)
rasterBasepr <- projectExtent(rasterBase, crs=CRS("+proj=longlat +ellps=WGS84 +datum=WGS84"))  
writeRaster(rasterBasepr, filename="rasterBaseAggpr.tif",format="GTiff", overwrite=TRUE)

# reproject to longlat while keeping values (projectRaster), with rasterBasepr as a template (to keep the same dimensions, resolution and extent)
for (c in 1:nrow(matveg)) {
#c=2
    combveg <- names(matveg[, matveg[c,]=="yes", drop=FALSE]) 
    combname <- paste(combveg, collapse="")
    namerastervegAgg <- paste("raster", combname, "Agg", sep="")
    namerastervegAggtif <- paste(namerastervegAgg, ".tif", sep="") 
    rasterveg <- raster(namerastervegAggtif)	
    proj4string(rasterveg) <- CRS("+proj=utm +zone=38 +south +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0")  		# original data in UTM
    pr_rasterveg <- projectRaster(rasterveg, rasterBasepr, crs=CRS("+proj=longlat +ellps=WGS84 +datum=WGS84"), method = "ngb")
    namerastervegAggprtif <- paste(namerastervegAgg, "pr.tif", sep="")
    writeRaster(pr_rasterveg, filename=namerastervegAggprtif,format="GTiff", overwrite=TRUE)
}

<br>
### Lemurs

##### Lemur traits (max width of seeds dispersed by each disperser)

In [ ]:
traitdisp <- read.csv("Dispersers.csv", sep=";")

## For this spatial analysis, we work only on lemurs (i.e. primates)
lemur <- subset(traitdisp, Taxon=="Primate")
lemur$Disperser = factor(lemur$Disperser)
lemursp <- lemur$Disperser

rasterForBushNA[rasterForBushNA==0] <- NA

<br>
###### Create rasters for each lemur species

In [ ]:
listrasterlem <- list()
arealem <- list()

lem <- data.frame(NAsrasterlem=rep(NA,length(lemursp)), NAsbuf=rep(NA,length(lemursp)), DiffNAs=rep(NA,length(lemursp)), Area=rep(NA,length(lemursp)))
row.names(lem) <- lemursp

rasterBase <- rasterBasepr

for (l in seq_along(lemursp)) {
	namefile <- paste("F:/Boulot/4-Kew/Data/Analyses/Spatial/Lemurs/", lemursp[l], ".csv", sep="")

	### Transform points into a raster object
	distr <- read.csv(file=namefile, sep=";")
	d <- rasterize(distr, rasterBase, field=1)

	### Create 5000-m buffer circles around each lemur cell and merge overlapping buffers
	buf <- buffer(d, width=5000)                              # create a new Raster object

	### Keep only forest/bush areas in the raster (lemurs live only in forests or bushes; to remove points that are artefacts or too old to represent the current distribution of lemurs, we remove any points that are not in a forest or bush area)
	rasterlem <- mask(buf, rasterForBushNA)
	listrasterlem[[l]] <- rasterlem

	### Calculate area of the raster
	areacells <- area(rasterlem)                               # area of each cell in the raster
	values(rasterlem)[is.na(values(rasterlem))] <- 0           # replace NAs by 0, otherwise the multiplication will not be computed
	values(areacells) <- values(areacells) * values(rasterlem) # multiply the cell areas with Raster object with presence/absence values
	lem[l,"Area"] <- cellStats(areacells , 'sum')              # potential distribution area of the lemur species l (in km²)
	rasterlem[rasterlem==0] <- NA

	### Save the raster
	namerasterlemtif <- paste("raster",lemursp[l], ".tif", sep="") 
	writeRaster(rasterlem, filename=namerasterlemtif,format="GTiff", overwrite=TRUE)
}

names(listrasterlem) <- lemursp
save(listrasterlem, file="listrasterlem.rda")
